In [1]:
#explore what the paper is saying
#describe all params exactly as given
#follow along and see where it leads

#watch stanford cs221 2019 mdp lecs


In [2]:
import numpy as np
import math

In [3]:
freq_agile_radar_versions = [[9 * math.pow(10, 9), 10* math.pow(10, 9)]]

interval = (freq_agile_radar_versions[0][1] - freq_agile_radar_versions[0][0])/64
print (interval)

hop_states = []
for i in range (0,64):
    
    hop_states.append(int(freq_agile_radar_versions[0][0]+ interval * i))

print (hop_states)
print (len(hop_states))


15625000.0
[9000000000, 9015625000, 9031250000, 9046875000, 9062500000, 9078125000, 9093750000, 9109375000, 9125000000, 9140625000, 9156250000, 9171875000, 9187500000, 9203125000, 9218750000, 9234375000, 9250000000, 9265625000, 9281250000, 9296875000, 9312500000, 9328125000, 9343750000, 9359375000, 9375000000, 9390625000, 9406250000, 9421875000, 9437500000, 9453125000, 9468750000, 9484375000, 9500000000, 9515625000, 9531250000, 9546875000, 9562500000, 9578125000, 9593750000, 9609375000, 9625000000, 9640625000, 9656250000, 9671875000, 9687500000, 9703125000, 9718750000, 9734375000, 9750000000, 9765625000, 9781250000, 9796875000, 9812500000, 9828125000, 9843750000, 9859375000, 9875000000, 9890625000, 9906250000, 9921875000, 9937500000, 9953125000, 9968750000, 9984375000]
64


In [4]:
# Equation:     JPr    =     ((PJ x GJ x GR x lam2 x BWR)     /     ((4 x pi)2 x R2J x BWJ x L))
#  JPr        = Jam Power at Receiver (Watt)
#  PJ         = Jammer Transmitted Power (Watt)
#  GJ         = Antenna Gain of Jammer (Figure)
#  GJR        = Antenna Gain of Radar (Figure)
#  lam       = Wavelength (m)
#  BWR     = IF Bandwidth Radar (Hz)
#  RJ         = Range to Jammer (m)
#  BWJ     = Jamming Bandwidth
#  L          = Losses
#  BWR and BWJ are matched no Losses!

In [5]:
jam_power = 1
g_j = 1
g_jr = 1
lambda_j = 1
bw_j = 1
bw_r = 1
r_j = 1
losses = 1

k = bw_r/(bw_j * losses)

#effective jam power at radar receiver

jam_power_eff = (jam_power * g_j * g_jr * np.power(lambda_j,2) * k)/(4 * np.pi * np.power(r_j, 2))
print (jam_power_eff)

transmitted_power = 1
g_t = 1 #antenna gain of transmitter
lambda_s = 1 #wavelength of signal
delta = 1
r_s = 1 #range to signal

#signal power at radar receiver

signal_power = (transmitted_power * np.power(g_t, 2) * np.power(lambda_s, 2) * delta)/(4 * np.power(np.pi, 3) * np.power(r_s, 4))
print(signal_power)

# k_aj is the suppression coeff
# when jam_power_eff/signal_power > k_aj, the jamming is effective

jsr = jam_power_eff/signal_power #jam to signal ratio

jsr_eff = jsr * bw_j/bw_r #effective jsr






0.07957747154594767
0.008062883608299874


In [6]:
import gymnasium as gym
from gym import Env
from gym import spaces
import random
import numpy as np
from IPython.display import clear_output
import pygame
import os

In [7]:
import gym
from gym import spaces
import numpy as np

class RadarJammingEnv(gym.Env):
    def __init__(self, hop_states, max_hop_length=32):
        super(RadarJammingEnv, self).__init__()

        # radar params
        self.p_j = 1 #power of jamming signal
        self.g_j = 1 #antenna gain of jammer
        self.l_pj = 1 #path loss of jamming signal from jammer to radar
        self.p_s = 1 #power of radar signal
        self.g_r = 1 #antenna gain of radar
        self.l_ps = 1 #path loss from radar to the target
        self.dist = 1 #distance bw the radar and the target
        self.bw_j = 1 #bandwidth of the jammer
        self.bw_r = 1 #bandwidth of the radar

        #all available freqs
        self.hop_states = hop_states
        self.n_frequencies = len(hop_states)

        #maximum num choices for the radar hopping sequence
        self.max_hop_length = max_hop_length
        
        #action space
        self.action_space = spaces.Box (low=hop_states[0], high=hop_states[-1], shape=(1,), dtype=np.float32)
        
        #observation space for current radar freq used
        self.observation_space = spaces.Box(low=0, high=1, shape=(self.n_frequencies,), dtype=np.float32)
        
        #init hopping patter
        self.hopping_pattern = self.generate_hopping_pattern ()
        self.current_step = 0
        
        #radar frequency used in this step
        self.current_frequency = self.next_frequency ()

    def calc_eff_jsr (self):

        eff_jsr = (self.p_j * self.g_j * self.l_pj * self.bw_j)/(self.p_s * self.g_r * self.l_ps * self.bw_r)
        return eff_jsr
        
    def generate_hopping_pattern (self):

        #random sequence of length max_hop_length hop states
        hopping_pattern = np.random.choice(self.hop_states, self.max_hop_length, replace=False)
        return hopping_pattern

    
    def next_frequency (self):

        return self.hopping_pattern[self.current_step % self.max_hop_length]
    
    def reset(self):

        self.hopping_pattern = self.generate_hopping_pattern ()
        self.current_frequency = self.next_frequency ()
        self.current_step = 0
        
        return self.get_observation ()
    
    def get_observation (self):

        # Observation is a one-hot encoded vector of the current radar frequency
        observation = np.zeros(self.n_frequencies)
        freq_index = np.where(self.hop_states == self.current_frequency)[0][0]
        observation[freq_index] = 1
        return observation
    
    def step(self, action):

        if self.current_step == 0:
            print ("Starting Simulation")
        
        done = False
        reward = 0

        # calc range within which the jammer is effective
        lower_bound = action - 100e6
        upper_bound = action + 100e6
        
        # check if the current radar frequency is within the jammer's range
        freq_index = np.where(self.hop_states == self.current_frequency)[0][0]
        print ("radar freq is: ", hop_states[freq_index])

        #this will store the correct frequency used in this step
        self.frequency_used = self.current_frequency
        
        if lower_bound <= self.hop_states[freq_index] <= upper_bound:

            reward = 1 

        else:
            reward = -1 

        self.current_step += 1
        #updating the frequency for the next step
        self.current_frequency = self.next_frequency ()
        
        #end episode
        if self.current_step >= 25:
            done = True
        
        return self.get_observation (), reward, done, {}
    
    def render(self, mode='human'):
        print(f"Step: {self.current_step},  Radar Frequency: {self.frequency_used},   Action: {action}")


In [8]:
hop_states = hop_states
env = RadarJammingEnv(hop_states)

# Example usage
obs = env.reset()
done = False

while not done:
    action = env.action_space.sample()
    print(f"Sampled Action: {action}")
    obs, reward, done, info = env.step(action)
    env.render()
    print(f"Action: {action}, Reward: {reward}")
    print ('\n')


Sampled Action: [9.330868e+09]
Starting Simulation
radar freq is:  9046875000
Step: 1,  Radar Frequency: 9046875000,   Action: [9.330868e+09]
Action: [9.330868e+09], Reward: -1


Sampled Action: [9.538328e+09]
radar freq is:  9796875000
Step: 2,  Radar Frequency: 9796875000,   Action: [9.538328e+09]
Action: [9.538328e+09], Reward: -1


Sampled Action: [9.575001e+09]
radar freq is:  9531250000
Step: 3,  Radar Frequency: 9531250000,   Action: [9.575001e+09]
Action: [9.575001e+09], Reward: 1


Sampled Action: [9.075539e+09]
radar freq is:  9343750000
Step: 4,  Radar Frequency: 9343750000,   Action: [9.075539e+09]
Action: [9.075539e+09], Reward: -1


Sampled Action: [9.419848e+09]
radar freq is:  9640625000
Step: 5,  Radar Frequency: 9640625000,   Action: [9.419848e+09]
Action: [9.419848e+09], Reward: -1


Sampled Action: [9.234991e+09]
radar freq is:  9828125000
Step: 6,  Radar Frequency: 9828125000,   Action: [9.234991e+09]
Action: [9.234991e+09], Reward: -1


Sampled Action: [9.808808e+